 
---

# Minimum Element in Stack in O(1) Space

## 1. IP–OP–PS (Input, Output, Problem Statement)

### **Problem Statement**

Design a **Stack** that supports the following operations in **O(1)** average time and **without extra space** for tracking the minimum:

1. **push(x)** – Push an element \(x\) onto the stack.  
2. **pop()** – Remove the top element.  
3. **top()** – Retrieve the top element.  
4. **getMin()** – Retrieve the **minimum** element in the stack at any time.

> **Note**: By “O(1) extra space,” we mean we are not allowed to use a second stack or additional data structure proportional to the number of elements. We can still store a **few** integer variables (e.g., for the current minimum).

### **Input**

- A sequence of operations on the stack, for example:
  ```
  push(3)
  push(5)
  getMin()
  push(2)
  push(2)
  getMin()
  pop()
  getMin()
  ...
  ```

### **Output**

- Results of **top()** or **getMin()** operations (and any push/pop side effects).
- The key requirement is **getMin()** must run in **O(1)** time without an auxiliary data structure storing multiple elements.

### **Detailed Example**

Consider this sequence:

1. `push(3)`  
   - Stack: [3], currentMin = 3  
2. `push(5)`  
   - Stack: [3, 5], currentMin = 3  
3. `getMin()` → 3  
4. `push(2)`  
   - **2 < currentMin** (3), so we **encode** this push and update currentMin = 2.  
   - Internally, we push a “special value” (e.g., `2*2 - 3 = 1`) to represent 2.  
   - Stack might look like [3, 5, 1], but effectively the top is “2.”  
5. `push(2)` (again)  
   - Now currentMin = 2, new value = 2, which is **equal** to currentMin.  
   - We can push 2 normally or encode again. Some variants encode again.  
   - Let’s push normally for clarity.  
   - Stack: [3, 5, 1, 2], currentMin = 2  
6. `getMin()` → 2  
7. `pop()`  
   - Popping top = 2. Since 2 == currentMin, we do not decode anything if we pushed it normally. currentMin remains 2.  
   - Stack: [3, 5, 1], currentMin = 2  
8. `pop()`  
   - Now the top = 1. This is the **encoded value** that represents the old minimum.  
   - Because top < currentMin, we decode the old min:  
     - originalMin = currentMin  
     - currentMin = `2*originalMin - top` = `2*2 - 1 = 3`.  
   - Stack after pop: [3, 5], currentMin = 3  
9. `getMin()` → 3  

At the end, the **minimum** in the stack is 3, with the stack storing `[3, 5]`.

---

## 2. Identification

### **Why This Approach (No Extra Stack)?**

1. **Auxiliary Stack Method**  
   - Typically, we keep a second stack (`minSt`) for O(1) retrieval of the minimum, but that uses up to O(n) **extra space**.

2. **Key Cues**  
   - We only need to track the **current minimum** and how to restore the previous minimum when that minimum is popped.

3. **Encoding Strategy**  
   - **When pushing** an element **x**:  
     - If `x >= currentMin`, push it normally.  
     - If `x < currentMin`, push an **encoded** value = `2*x - currentMin`, and set `currentMin = x`.  
   - **When popping**:  
     - If the popped element is **less than currentMin**, it’s an **encoded** value. Decode to find the old min:  
       \[
       \text{oldMin} = 2 \times \text{currentMin} - \text{encodedVal}
       \]
       Then restore `currentMin = oldMin`.

4. **Result**  
   - `getMin()` is always **O(1)** because we store `currentMin` in a variable.  
   - We do not need an entire second data structure.

---

## 3. Break Down → Encoded Push/Pop Logic

### **Step-by-Step Sub-Tasks**

1. **Initialize**  
   - Create an empty stack `st`.  
   - `currentMin` = some sentinel (e.g., set it to `INT_MAX` after the first push or handle the first element as a special case).

2. **Push(x)**  
   - If the stack is empty, `currentMin = x`, push `x` directly.  
   - Else:  
     - If `x >= currentMin`: push `x` normally.  
     - If `x < currentMin`:  
       - **Encode** the value to store in the stack: `encodedVal = 2*x - currentMin`.  
       - Push `encodedVal` into `st`.  
       - Update `currentMin = x`.

3. **Pop()**  
   - Pop the top element, call it `topVal`.  
   - If `topVal >= currentMin`, return `topVal` as normal.  
     - (No min update needed, since it’s not an encoded value or it’s bigger than the current min.)  
   - Else (means `topVal` < `currentMin`, i.e. it’s an encoded value):  
     - Decode the old minimum: `oldMin = 2*currentMin - topVal`.  
     - Update `currentMin = oldMin`.  
     - Return the “real popped value,” which was the old min. (Or keep track if needed.)

4. **top()**  
   - If the top of the stack is **>= currentMin**, that top is the real value.  
   - If the top is **< currentMin**, it’s an encoded value. The real top is `currentMin`.

5. **getMin()**  
   - Return `currentMin` in **O(1)**.

### **Data Structures Used**

- A single **stack** of integers, storing either the real value (if >= currentMin) or an encoded value (if < currentMin).  
- An **integer variable** `currentMin` to keep track of the current minimum.

---

## 4. Explanations + Code

### **Detailed Explanation**

1. **Encoding**  
   - When pushing a value smaller than the current minimum, we store `2*x - currentMin` in the stack.  
   - This ensures we can later decode the previous min when popping.

2. **Decoding**  
   - When popping, if the popped value is **less than** `currentMin`, it indicates we had pushed an encoded value.  
   - We decode the old minimum by `oldMin = 2*currentMin - encodedVal`. This recovers the minimum before we updated it.

3. **Time Complexity**  
   - All operations (`push`, `pop`, `top`, `getMin`) run in **O(1)** average time.  
   - We store only one integer per element in the main stack (no extra stack), so the **auxiliary space** is O(1).

### **C++ Implementation**

```cpp
#include <iostream>
#include <stack>
#include <stdexcept>
#include <climits>
using namespace std;

class MinStackNoExtraSpace {
private:
    stack<long long> st;  // main stack to store actual/encoded values
    long long currMin;    // tracks current minimum

public:
    MinStackNoExtraSpace() {
        // Optionally set currMin = LLONG_MAX or handle empty stack carefully
        currMin = LLONG_MAX;
    }

    void push(long long x) {
        if(st.empty()) {
            // First element
            st.push(x);
            currMin = x;
        } else {
            if(x >= currMin) {
                // Just push x normally
                st.push(x);
            } else {
                // Encode the value
                long long encodedVal = 2LL * x - currMin;
                st.push(encodedVal);
                currMin = x;  // update current min
            }
        }
    }

    void pop() {
        if(st.empty()) {
            throw runtime_error("Stack is empty, cannot pop.");
        }
        long long topVal = st.top();
        st.pop();

        if(topVal < currMin) {
            // topVal is encoded; decode old min
            long long oldMin = 2LL * currMin - topVal;
            currMin = oldMin;
        }
    }

    long long top() {
        if(st.empty()) {
            throw runtime_error("Stack is empty, no top element.");
        }
        long long topVal = st.top();
        // If topVal < currMin, it's an encoded value => real top is currMin
        if(topVal < currMin) {
            return currMin; 
        } else {
            return topVal;
        }
    }

    long long getMin() {
        if(st.empty()) {
            throw runtime_error("Stack is empty, no minimum.");
        }
        return currMin;
    }

    bool empty() {
        return st.empty();
    }
};

int main() {
    MinStackNoExtraSpace st;
    st.push(3);
    st.push(5);
    cout << "Current min: " << st.getMin() << endl; // 3
    st.push(2);
    st.push(2);
    cout << "Current min: " << st.getMin() << endl; // 2
    cout << "Top element: " << st.top() << endl;    // 2
    st.pop(); // pops 2
    cout << "Current min: " << st.getMin() << endl; // still 2
    st.pop(); // pops the encoded value => old min restored
    cout << "Current min: " << st.getMin() << endl; // 3
    return 0;
}
```

---

## 5. Animated Visualization

Below is a **Python** snippet using `matplotlib` and `ipywidgets` to illustrate how this **encoded** approach works step by step. It shows the stack’s content (with possible encoded values) and the `currentMin` after each operation.

```python
import matplotlib.pyplot as plt
from ipywidgets import interact, IntSlider

def min_stack_no_extra_space_visual(step_index):
    operations = [
        ("push", 3),
        ("push", 5),
        ("getMin", None),
        ("push", 2),
        ("push", 2),
        ("getMin", None),
        ("top", None),
        ("pop", None),
        ("getMin", None),
        ("pop", None),
        ("getMin", None),
    ]
    
    st = []
    currMin = None
    states = []
    
    def push_val(x):
        nonlocal currMin
        if not st:  # empty
            st.append(x)
            currMin = x
        else:
            if x >= currMin:
                st.append(x)
            else:
                encodedVal = 2*x - currMin
                st.append(encodedVal)
                currMin = x
    
    def pop_val():
        nonlocal currMin
        if not st:
            return None
        topVal = st.pop()
        if topVal < currMin:
            # decode
            oldMin = 2*currMin - topVal
            poppedReal = currMin
            currMin = oldMin
            return poppedReal
        else:
            return topVal
    
    def get_min():
        return currMin
    
    def get_top():
        if not st:
            return None
        topVal = st[-1]
        if topVal < currMin:
            return currMin
        else:
            return topVal
    
    for i, (op, val) in enumerate(operations):
        state = {
            'step': i+1,
            'operation': f"{op}({val})" if val is not None else op,
            'stack_before': st.copy(),
            'currMin_before': currMin,
            'return_val': None
        }
        
        # Perform operation
        if op == "push":
            push_val(val)
        elif op == "pop":
            popped = pop_val()
            state['return_val'] = popped
        elif op == "getMin":
            state['return_val'] = get_min()
        elif op == "top":
            state['return_val'] = get_top()
        
        state['stack_after'] = st.copy()
        state['currMin_after'] = currMin
        states.append(state)
    
    if step_index >= len(states):
        step_index = len(states) - 1
    step = states[step_index]
    
    fig, ax = plt.subplots(figsize=(8,4))
    ax.set_title(f"Step {step['step']}: {step['operation']}")
    ax.set_xlim(0, 2)
    ax.set_ylim(0, 10)
    ax.set_xticks([])
    ax.set_yticks([])
    
    # Display stack before/after
    stack_before_str = "\n".join(str(x) for x in reversed(step['stack_before']))
    stack_after_str = "\n".join(str(x) for x in reversed(step['stack_after']))
    
    annotation = (f"Stack before:\n{stack_before_str}\n\n"
                  f"currMin before: {step['currMin_before']}\n"
                  f"Operation return: {step['return_val']}\n\n"
                  f"Stack after:\n{stack_after_str}\n\n"
                  f"currMin after: {step['currMin_after']}")
    ax.text(0.02, 0.95, annotation, fontsize=10,
            bbox=dict(facecolor='white', alpha=0.7), verticalalignment='top',
            transform=ax.transAxes)
    
    plt.show()

interact(min_stack_no_extra_space_visual, step_index=IntSlider(min=0, max=10, step=1, value=0));
```

### **How to Use This Visualization**

1. Copy the snippet into a Jupyter Notebook cell.  
2. Run the cell.  
3. Use the **slider** to step through each operation.  
4. The “stack” displayed may contain **encoded values** (shown as integers that might be less than `currMin`).  
5. The **annotation** shows the state of the stack before and after, the `currMin` before and after, and any **return value** (e.g., from `pop()`, `getMin()`, or `top()`).

---

# Summary

1. **Problem Statement (IP–OP–PS):**  
   - Implement a stack supporting `push`, `pop`, `top`, and `getMin` in O(1) time **without extra space** for storing minimums.

2. **Identification:**  
   - We cannot use a second stack (like the auxiliary approach). Instead, we encode values in the main stack.

3. **Breakdown (Encoding Strategy):**  
   - **Push(x)**: If `x < currentMin`, push `2*x - currentMin` and update `currentMin`. Otherwise, push `x`.  
   - **Pop()**: If popped value < `currentMin`, decode to get old minimum.  
   - **top()**: If top < `currentMin`, the real top is `currentMin`.  
   - **getMin()**: Return `currentMin`.

4. **Explanations + Code:**  
   - A detailed C++ solution demonstrates O(1) time and O(1) extra space.  
   - All data is stored in a single stack with possible “encoded” values.

5. **Animated Visualization:**  
   - The Python snippet uses `matplotlib` + `ipywidgets` to illustrate each operation’s effect on the stack and `currentMin`.
 